## A simple analytical PointNetLK example

Load packages.

In [1]:
# !pip install git+git://github.com/Lilac-Lee/PointNetLK_Revisited

import argparse
import os
import sys
import numpy as np
import torch
import torch.utils.data
import torchvision

# visualize the point cloud
import open3d as o3d
# open3d>=0.13.0, otherwise, comment line below
o3d.visualization.webrtc_server.enable_webrtc()

sys.path.insert(0, '../')
import data_utils
import trainer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] Resetting default logger to print to terminal.


Initialize some testing and network parameters as arguments.

In [2]:
args = argparse.Namespace()

# dimension for the PointNet embedding
args.dim_k = 1024

# device: cuda/cpu
# args.device = 'cuda:0'
args.device = 'cpu'

# maximum iterations for the LK
args.max_iter = 120

# embedding function: pointnet
args.embedding = 'pointnet'

# output log file name
args.outfile = 'toyexample_2021_04_17'

# specify data type: real
args.data_type = 'real'

# specify visualize result or not
args.vis = True

Get toy example point cloud pairs, and ground truth rigid pose.
Set some voxelization related parameters.

In [3]:
# load data
p0 = np.load('./p0.npy')[np.newaxis,...]
p1 = np.load('./p1.npy')[np.newaxis,...]

# randomly set the twist parameters for the ground truth pose
x = np.array([[0.57, -0.29, 0.73, -0.37, 0.48, -0.54]])

# set voxelization parameters
voxel_ratio = 0.05
voxel = 2
max_voxel_points = 1000
num_voxels = 8

# construct the testing dataset
testset = data_utils.ToyExampleData(p0, p1, voxel_ratio, voxel, max_voxel_points, num_voxels, x, args.vis)

Create the model class, load the pre-trained model, and begin testing!

In [ ]:
# create model
dptnetlk = trainer.TrainerAnalyticalPointNetLK(args)
model = dptnetlk.create_model()

# specify device
if not torch.cuda.is_available():
    args.device = 'cpu'
args.device = torch.device(args.device)
model.to(args.device)

# load pre-trained model
model.load_state_dict(torch.load('../logs/model_trained_on_ModelNet40_model_best.pth', map_location='cpu'))

# testloader
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=0, drop_last=False)

# begin testing
dptnetlk.test_one_epoch(model, testloader, args.device, 'test', args.data_type, args.vis, toyexample=True)

  0%|                                              | 0/1 [00:00<?, ?it/s]